In [1]:
#!/usr/bin/env python3

"""
Trying to outline out a basic transaction 
in a notebook
"""

'\nTrying to outline out a basic transaction \nin a notebook\n'

In [2]:
"""
basic imports of standard libraries,
and finding and importing provided libraries,
"""
import os, sys

if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
    print(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

if(os.sep == '/'):
    from lib.sign import sign_tx
else:
    from lib.sign_win import sign_tx
from lib.encoder  import encode_tx, encode_script
from lib.helper   import decode_address
from lib.hash     import hash160, hash256, sha256
from lib.rpc      import RpcSocket

C:\Users\zhvhe\Documents\GitHub\blockchain\python


In [3]:
## Setup our RPC socket.
rpc = RpcSocket({ 'wallet': 'test2' })
assert rpc.check()

In [4]:
## First, we will lookup an existing utxo,
## and use that to fund our transaction.
utxo = rpc.get_utxo(0)
print(utxo)

{'address': 'tb1qldgv444r8705w4seuvcsx7nz8sq5srguxxwfxl', 'priv_key': '45880cda3de2237b658aabeb4480cd8c5882b8e50085fadf21cb12f41c3885ae', 'pub_key': '03bcdb6ad6b88740b0dc0debd6f78eb1ea6b0f0ad98d62b1b4347794fe9f3415ff', 'pubkey_hash': 'fb50cad6a33f9f475619e331037a623c01480d1c', 'redeem_script': '1976a914fb50cad6a33f9f475619e331037a623c01480d1c88ac', 'txid': 'f754c8478fc32a36f6f990b077f215103df72fc2da6cc177006441cabbc2800c', 'vout': 0, 'value': 1582806}


In [5]:
## We will also grab a new receiving address,
## and lock the funds to this address.
recv = rpc.get_recv()
print(recv)

{'address': 'tb1qr8dq6jrwz69cqjw9gsrhexh3dj39xxmttg5264', 'priv_key': '656c99aabf99b7d08d18c408045d5bb7e0582f071c8db795612d3bc7f7699d0c', 'pub_key': '0251de3a4197e034bb34f4c01980fd368b886f707f43fa3f3beda1e1d0e111438e', 'pubkey_hash': '19da0d486e168b8049c544077c9af16ca2531b6b', 'redeem_script': '1976a91419da0d486e168b8049c544077c9af16ca2531b6b88ac'}


In [6]:
## Replace this default preimage with your own secret.
secret_preimage = 'weareallsatoshi'

## Convert the secret to bytes, then hash using hash160 function.
secret_bytes = secret_preimage.encode('utf8').hex()
secret_hash  = hash160(secret_bytes).hex()

In [7]:
## This is where we specify the version number for the program interpreter. 
## We'll be using version 0.
script_version = 0

## Here is the locking script that we will be using. We are going to
## require the redeemer to reveral the secret, along with their public
## key for the receipt address, and matching signature.
script_words = [
    'OP_HASH160', secret_hash, 'OP_EQUALVERIFY', 
    'OP_DUP', 'OP_HASH160', hash160(recv['pub_key']).hex(), 'OP_EQUALVERIFY', 
    'OP_CHECKSIG'
]

In [8]:
## This is the hex-encoded format of the script. We will present this when 
## we unlock and spend the output. It should match the pre-image used for 
## making the script hash.
redeem_script = encode_script(script_words, prepend_len=False).hex()

## We hash the script using sha256, then provide a version number
## along with the hash. This will lock the transaction output to 
## accept only the program script which matches the hash.
script_hash = sha256(redeem_script).hex()

## Calculate the value of the transaction output, minus fees.
locking_tx_value = utxo['value'] - 1000

In [9]:
## The initial locking transaction. This spends the utxo from our funding 
## transaction, and moves the funds to the utxo for our witness program.
locking_tx = {
    'version': 1,
    'vin': [{
        'txid': utxo['txid'],
        'vout': utxo['vout'],
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [{
        'value': locking_tx_value,
        'script_pubkey': [ script_version, script_hash ]
    }],
    'locktime': 0
}

In [10]:
## Encode the transaction into raw hex,
## and calculate the transaction ID
locking_hex  = encode_tx(locking_tx)
locking_txid = hash256(bytes.fromhex(locking_hex))[::-1].hex()

In [11]:
## Sign the transaction using our key-pair from the utxo.
locking_sig = sign_tx(
  locking_tx,               # The transaction.
  0,                        # The input being signed.
  utxo['value'],            # The value of the utxo being spent.
  utxo['pubkey_hash'],
  utxo['priv_key']          # The private key to the utxo pubkey hash.
)

In [12]:
## Add the signature and public key to the transaction.
locking_tx['vin'][0]['witness'] = [ locking_sig, utxo['pub_key'] ]

In [13]:
print(f'''
# Pay-to-Witness-Script-Hash Example

Locking Txid:
{locking_txid}

Redeem Script:
{redeem_script}

Locking Tx:
{encode_tx(locking_tx)}
''')


# Pay-to-Witness-Script-Hash Example

Locking Txid:
085c98047e7abe778c39dde19d54bc11ebe98a9717e663a23bcd63a16a50fdd7

Redeem Script:
a91478d574f6f76809749cb351af7e20ae52082e2e378876a91419da0d486e168b8049c544077c9af16ca2531b6b88ac

Locking Tx:
010000000001010c80c2bbca41640077c16cdac22ff73d1015f277b090f9f6362ac38f47c854f70000000000ffffffff01ee2218000000000022002082525f9f4512e25bea993b5b9c22d43e09d5c2cae083b2b9303a3802af13b14b02473044022075d7ed2775030f87b73ecf129127d5953660cd3acc2e4b5a93967db703468f7c0220082dbe831cbce0a0f16425a95dee6a4e4f3e86c6342a20a336a7c69f5810efd5012103bcdb6ad6b88740b0dc0debd6f78eb1ea6b0f0ad98d62b1b4347794fe9f3415ff00000000



In [14]:
## send it off :D
rpc.transact(encode_tx(locking_tx))

'085c98047e7abe778c39dde19d54bc11ebe98a9717e663a23bcd63a16a50fdd7'

In [15]:
## Bech32 addresses will decode into a script version and pubkey hash.
script_version, pubkey_hash = decode_address(recv['address'])

In [16]:
## This transaction will redeem the previous utxo by providing the secret 
## pre-image, plus the public key and signature, plus the witness program. 
## Once the transaction is confirmed, your wallet software should recognize 
## this utxo as spendable.
redeem_tx = {
    'version': 1,
    'vin': [{
        'txid': locking_txid,
        'vout': 0,
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [{
        'value': locking_tx_value - 1000,
        'script_pubkey': [ script_version, pubkey_hash ]
    }],
    'locktime':0
}

redeem_sig = sign_tx(
  redeem_tx,
  0,
  locking_tx_value,
  redeem_script,
  recv['priv_key']
)

In [17]:
redeem_tx['vin'][0]['witness'] = [ redeem_sig, recv['pub_key'], secret_bytes, redeem_script ]

print(f'Unlocking Tx:\n{encode_tx(redeem_tx)}')

Unlocking Tx:
01000000000101d7fd506aa163cd3ba263e617978ae9eb11bc549de1dd398c77be7a7e04985c080000000000ffffffff01061f18000000000016001419da0d486e168b8049c544077c9af16ca2531b6b044830450221008cf335fb2ee4275649d91e49743e7009bf155eb6d0e20707e366e8a7876a42c20220485ea0ec7a962080b8f464d3aaa7f8d362bda8c16891a90b5e19013577643a1601210251de3a4197e034bb34f4c01980fd368b886f707f43fa3f3beda1e1d0e111438e0f7765617265616c6c7361746f73686930a91478d574f6f76809749cb351af7e20ae52082e2e378876a91419da0d486e168b8049c544077c9af16ca2531b6b88ac00000000


In [18]:
## redeem it
rpc.transact(encode_tx(redeem_tx))

'cc3f42167a6c7ba24151ba3031a826a38cf601a6450285db03b1b7274b22e618'